In [1]:
##########################################
# Pricing/Mispricing Paper Table 2       #
# Andrew Lou                             #
# Date: June 28 2022                     #
##########################################

# package imports
from tkinter import Y
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import matplotlib.pyplot as plt
import getFamaFrenchFactors as gff
import statsmodels.api as smf
from pandas.tseries.offsets import *
from scipy import stats

# connect to WRDS
conn = wrds.Connection()

crsp_m = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date >='01/01/1968'
                      and b.exchcd between 1 and 3
                      """, date_cols=['date'])

Enter your WRDS username [andrewlou]: alou6683
Enter your password: ············


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
Loading library list...
Done


In [2]:
# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']]=crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['jdate']=crsp_m['date']+MonthEnd(0)

# add delisting return
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """, date_cols=['dlstdt'])

In [3]:
dlret.permno=dlret.permno.astype(int)
#dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+MonthEnd(0)

crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])
crsp['dlret']=crsp['dlret'].fillna(0)
crsp['ret']=crsp['ret'].fillna(0)

# retadj factors in the delisting returns
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1

# calculate market equity
crsp['me']=crsp['prc'].abs()*crsp['shrout'] 
crsp=crsp.drop(['dlret','dlstdt','prc','shrout'], axis=1)
crsp=crsp.sort_values(by=['jdate','permco','me'])

In [4]:
crsp.head(5)

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me
945326,28820,6,1968-01-31,10,2,-0.144385,-0.144385,1968-01-31,-0.144385,57000.0
953346,29161,64,1968-01-31,10,2,0.551020,0.551020,1968-01-31,0.551020,20900.0
572378,17670,74,1968-01-31,10,1,-0.017857,-0.017857,1968-01-31,-0.017857,34375.0
1268572,41515,80,1968-01-31,11,1,0.119266,0.119266,1968-01-31,0.119266,177876.0
620508,18702,267,1968-01-31,10,1,0.142857,0.142857,1968-01-31,0.142857,48992.0


In [5]:
### Aggregate Market Cap ###
# sum of me across different permno belonging to same permco a given date
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()

# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()

# join by jdate/maxme to find the permno
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])

# drop me column and replace with the sum me
crsp1=crsp1.drop(['me'], axis=1)

# join with sum of me to get the correct market cap info
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])

# sort by permno and date and also drop duplicates
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()

crsp2['year']=crsp2['jdate'].dt.year
crsp2['month']=crsp2['jdate'].dt.month

crsp2.head(80)

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,year,month
989454,10000,7952,1986-01-31,10,3,0.000000,NaN,1986-01-31,0.000000,16100.000000,1986,1
995656,10000,7952,1986-02-28,10,3,-0.257143,-0.257143,1986-02-28,-0.257143,11960.000000,1986,2
1001867,10000,7952,1986-03-31,10,3,0.365385,0.365385,1986-03-31,0.365385,16330.000000,1986,3
1008088,10000,7952,1986-04-30,10,3,-0.098592,-0.098592,1986-04-30,-0.098592,15172.000000,1986,4
1014319,10000,7952,1986-05-30,10,3,-0.222656,-0.222656,1986-05-31,-0.222656,11793.859375,1986,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1378307,10001,7953,1990-11-30,11,3,0.000000,0.000000,1990-11-30,0.000000,10048.500000,1990,11
1384881,10001,7953,1990-12-31,11,3,0.001299,-0.012987,1990-12-31,0.001299,10013.000000,1990,12
1391439,10001,7953,1991-01-31,11,3,0.013158,0.013158,1991-01-31,0.013158,10144.750000,1991,1
1397973,10001,7953,1991-02-28,11,3,0.012987,0.012987,1991-02-28,0.012987,10276.500000,1991,2


In [6]:
crsp2['1+retx']=1+crsp2['retx']
crsp2=crsp2.sort_values(by=['permno','date'])

# cumret by stock
crsp2['cumretx']=crsp2.groupby(['permno','year'])['1+retx'].cumprod()

# lag cumret
crsp2['lcumretx']=crsp2.groupby(['permno'])['cumretx'].shift(120)

# lag market cap
crsp2['lme']=crsp2.groupby(['permno'])['me'].shift(120)

# if first permno then use me/(1+retx) to replace the missing value
crsp2['count']=crsp2.groupby(['permno']).cumcount()
crsp3 = crsp2.copy()
crsp3 = crsp3.dropna()
crsp3['wt']=np.where(crsp3['jdate'], crsp3['lme'], crsp3['me']*crsp3['lcumretx'])
crsp3.head(20)

,permno,permco,date,shrcd,exchcd,ret,retx,jdate,retadj,me,year,month,1+retx,cumretx,lcumretx,lme,count,wt
1839811,10001,7953,1996-02-29,11,3,0.013699,0.013699,1996-02-29,0.013699,21099.250000,1996,2,1.013699,0.986667,1.020408,6156.2500,121,6156.2500
1848174,10001,7953,1996-03-29,11,3,0.036149,0.025338,1996-03-31,0.036149,21899.421875,1996,3,1.025338,1.011667,1.030612,6217.8125,122,6217.8125
1856604,10001,7953,1996-04-30,11,3,-0.070840,-0.070840,1996-04-30,-0.070840,20348.062500,1996,4,0.929160,0.940000,1.040816,6279.3750,123,6279.3750
1865092,10001,7953,1996-05-31,11,3,-0.021277,-0.021277,1996-05-31,-0.021277,19915.125000,1996,5,0.978723,0.920000,1.030612,6217.8125,124,6217.8125
1873656,10001,7953,1996-06-28,11,3,-0.060290,-0.072464,1996-06-30,-0.060290,18568.000000,1996,6,0.927536,0.853333,1.000000,6033.1250,125,6033.1250
1882299,10001,7953,1996-07-31,11,3,0.023438,0.023438,1996-07-31,0.023438,19003.187500,1996,7,1.023438,0.873333,0.989796,5971.5625,126,5971.5625
1890971,10001,7953,1996-08-30,11,3,0.038168,0.038168,1996-08-31,0.038168,19728.500000,1996,8,1.038168,0.906667,1.061224,6402.5000,127,6402.5000
1899694,10001,7953,1996-09-30,11,3,0.041765,0.029412,1996-09-30,0.041765,20527.500000,1996,9,1.029412,0.933333,1.040816,6317.6250,128,6317.6250
1908441,10001,7953,1996-10-31,11,3,-0.028571,-0.028571,1996-10-31,-0.028571,19941.000000,1996,10,0.971429,0.906667,1.081633,6565.3750,129,6565.3750
1917258,10001,7953,1996-11-29,11,3,0.029412,0.029412,1996-11-30,0.029412,20527.500000,1996,11,1.029412,0.933333,1.142857,6937.0000,130,6937.0000


In [7]:
anomaly = pd.read_parquet("anomaly.gzip")
# lag cumret
anomaly['lperf']=anomaly.groupby(['permno'])['perf'].shift(120)
# lag market cap
anomaly['lmgmt']=anomaly.groupby(['permno'])['mgmt'].shift(120)
fullanomaly = anomaly.copy()
fullanomaly = fullanomaly.dropna()
fullanomaly['yyyymm'] = fullanomaly['yyyymm'].astype(int)
fullanomaly.head(20)

,permno,yyyymm,RET,lag_prc,mktcap,EXCHCD,perf,mgmt,perf_cnt,mgmt_cnt,lperf,lmgmt
340660,10321.0,196202,0.069919,15.375,277626.375,1.0,5.624196,47.983946,4,3,30.530715,64.026779
340667,10487.0,196202,-0.020863,34.750,97821.250,1.0,7.207011,35.571858,4,3,68.757312,38.606889
340725,11607.0,196202,0.012048,41.500,207873.500,1.0,10.353684,35.962125,4,3,51.737379,32.670724
340768,12511.0,196202,0.128342,46.750,121643.500,1.0,66.395704,48.904145,4,3,64.477389,33.637636
340790,12976.0,196202,0.070140,62.375,123627.250,1.0,68.166242,34.974973,4,3,41.167547,58.488854
340884,15229.0,196202,-0.068182,33.000,33396.000,1.0,17.961905,65.949662,4,3,59.535357,45.242241
340937,16555.0,196202,0.066327,49.000,58506.000,1.0,69.396363,48.392420,4,3,40.073892,46.155051
340990,17670.0,196202,-0.084158,50.500,48227.500,1.0,36.629760,40.210739,4,3,43.487995,71.578477
341035,18438.0,196202,0.022727,22.000,13794.000,1.0,29.354875,26.693498,4,3,46.300928,33.613110
341078,19297.0,196202,-0.013453,27.875,41478.000,1.0,46.640861,58.186361,4,3,38.902486,42.085543


In [8]:
fullanomaly['date'] = pd.to_datetime(fullanomaly['yyyymm'], format='%Y%m')
fullanomaly['jdate'] = fullanomaly['date']+MonthEnd(0)
fullanomaly['jdate'] = fullanomaly['jdate'].dt.date
fullanomaly['jdate'] = fullanomaly['jdate'].astype('<M8[ns]')
fullanomaly['permno'] = fullanomaly['permno'].astype(int)
fullanomaly.sort_values(by=['permno']).head(120)

,permno,yyyymm,RET,lag_prc,mktcap,EXCHCD,perf,mgmt,perf_cnt,mgmt_cnt,lperf,lmgmt,date,jdate
2533497,10001,200307,0.018303,6.010,15595.950594,3.0,45.027902,59.275263,6,5,54.611554,43.722195,2003-07-01,2003-07-31
3209024,10001,201706,0.023622,12.700,133603.997993,2.0,49.469647,40.316378,6,5,37.828990,43.481581,2017-06-01,2017-06-30
2969011,10001,201111,-0.005000,11.000,89672.000000,2.0,93.649635,49.751782,6,5,59.616618,50.446246,2011-11-01,2011-11-30
2428465,10001,200111,-0.004329,11.550,29082.900480,3.0,59.616618,50.446246,6,5,56.124933,57.053717,2001-11-01,2001-11-30
3212518,10001,201707,0.001934,12.925,135971.002007,2.0,49.071081,38.787884,6,5,37.768290,42.133881,2017-07-01,2017-07-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3105193,10001,201501,-0.115245,11.020,115577.764801,2.0,69.803859,52.632019,6,5,54.251973,78.692067,2015-01-01,2015-01-31
3048517,10001,201309,0.004412,10.200,105794.398022,2.0,71.143054,69.471089,6,5,44.733148,63.979177,2013-09-01,2013-09-30
2204354,10001,199811,0.020270,9.250,22264.750000,3.0,36.421167,52.076578,6,5,37.535867,61.120090,1998-11-01,1998-11-30
2786831,10001,200802,0.021704,14.000,40250.000000,3.0,17.542745,46.040445,6,5,62.313534,67.225216,2008-02-01,2008-02-29


In [9]:
data = pd.merge(crsp3, fullanomaly, how='inner', on=['permno', 'jdate'])
data.head(20)

,permno,permco,date_x,shrcd,exchcd,ret,retx,jdate,retadj,me,...,lag_prc,mktcap,EXCHCD,perf,mgmt,perf_cnt,mgmt_cnt,lperf,lmgmt,date_y
0,10001,7953,1997-11-28,11,3,-0.014085,-0.014085,1997-11-30,-0.014085,20807.5000,...,8.8750,21104.7500,3.0,62.429241,68.594130,6,5,35.014919,63.506012,1997-11-01
1,10001,7953,1997-12-31,11,3,0.041143,0.028571,1997-12-31,0.041143,21555.0000,...,8.7500,20807.5000,3.0,62.478538,67.797886,6,5,34.940585,61.094271,1997-12-01
2,10001,7953,1998-01-30,11,3,0.000000,0.000000,1998-01-31,0.000000,21555.0000,...,9.0000,21555.0000,3.0,62.435050,67.927970,6,5,34.888327,62.866023,1998-01-01
3,10001,7953,1998-02-27,11,3,-0.006944,-0.006944,1998-02-28,-0.006944,21405.3125,...,9.0000,21555.0000,3.0,62.313534,67.225216,6,5,34.481144,63.597574,1998-02-01
4,10001,7953,1998-03-31,11,3,-0.008671,-0.020979,1998-03-31,-0.008671,21008.7500,...,8.9375,21405.3125,3.0,62.335992,56.435819,6,5,34.459162,62.375481,1998-03-01
5,10001,7953,1998-04-30,11,3,0.007143,0.007143,1998-04-30,0.007143,21158.8125,...,8.7500,21008.7500,3.0,62.283438,60.758889,6,5,34.607972,51.885226,1998-04-01
6,10001,7953,1998-05-29,11,3,-0.014184,-0.014184,1998-05-31,-0.014184,20858.6875,...,8.8125,21158.8125,3.0,62.122214,61.059661,6,5,34.399837,55.001974,1998-05-01
7,10001,7953,1998-06-30,11,3,0.006043,-0.007194,1998-06-30,0.006043,20725.8750,...,8.6875,20858.6875,3.0,62.043163,53.985115,6,5,34.576863,48.060428,1998-06-01
8,10001,7953,1998-07-31,11,3,0.014493,0.014493,1998-07-31,0.014493,21026.2500,...,8.6250,20725.8750,3.0,62.000302,52.489205,6,5,34.614709,46.458758,1998-07-01
9,10001,7953,1998-08-31,11,3,0.000000,0.000000,1998-08-31,0.000000,21026.2500,...,8.7500,21026.2500,3.0,62.013274,50.925627,6,5,34.932926,46.157684,1998-08-01


In [10]:
data=data[['permno', 'jdate', 'shrcd','exchcd','retadj','me','cumretx','mktcap','lme', 'lperf', 'lmgmt', 'perf', 'mgmt', 'count', 'wt']]
data=data.sort_values(by=['permno','jdate']).drop_duplicates()
data.head(20)


,permno,jdate,shrcd,exchcd,retadj,me,cumretx,mktcap,lme,lperf,lmgmt,perf,mgmt,count,wt
0,10001,1997-11-30,11,3,-0.014085,20807.5000,1.076923,21104.7500,6138.00,35.014919,63.506012,62.429241,68.594130,142,6138.00
1,10001,1997-12-31,11,3,0.041143,21555.0000,1.107692,20807.5000,5828.00,34.940585,61.094271,62.478538,67.797886,143,5828.00
2,10001,1998-01-31,11,3,0.000000,21555.0000,1.000000,21555.0000,6200.00,34.888327,62.866023,62.435050,67.927970,144,6200.00
3,10001,1998-02-28,11,3,-0.006944,21405.3125,0.993056,21555.0000,6696.00,34.481144,63.597574,62.313534,67.225216,145,6696.00
4,10001,1998-03-31,11,3,-0.008671,21008.7500,0.972222,21405.3125,6076.00,34.459162,62.375481,62.335992,56.435819,146,6076.00
5,10001,1998-04-30,11,3,0.007143,21158.8125,0.979167,21008.7500,6262.00,34.607972,51.885226,62.283438,60.758889,147,6262.00
6,10001,1998-05-31,11,3,-0.014184,20858.6875,0.965278,21158.8125,6386.00,34.399837,55.001974,62.122214,61.059661,148,6386.00
7,10001,1998-06-30,11,3,0.006043,20725.8750,0.958333,20858.6875,6200.00,34.576863,48.060428,62.043163,53.985115,149,6200.00
8,10001,1998-07-31,11,3,0.014493,21026.2500,0.972222,20725.8750,6386.00,34.614709,46.458758,62.000302,52.489205,150,6386.00
9,10001,1998-08-31,11,3,0.000000,21026.2500,0.972222,21026.2500,6572.00,34.932926,46.157684,62.013274,50.925627,151,6572.00


In [11]:
nyse=data[(data['exchcd']==1) & (data['lme']>0) & (data['lperf']>0) & \
             (data['count']>=1) & ((data['shrcd']==10) | (data['shrcd']==11))]
nyse_sz=nyse.groupby(['jdate'])['lme'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_sz=nyse_sz[['jdate','30%','70%']].rename(columns={'30%':'sz30', '70%':'sz70'})

nyse_perf=nyse.groupby(['jdate'])['lperf'].describe(percentiles=[0.3, 0.7]).reset_index()
nyse_perf=nyse_perf[['jdate','30%','70%']].rename(columns={'30%':'perf30', '70%':'perf70'})

nyse_breaks = pd.merge(nyse_sz, nyse_perf, how='inner', on=['jdate'])

data1 = pd.merge(data, nyse_breaks, how='left', on=['jdate'])

# function to assign sz and bm bucket
def sz_bucket(row):
    if row['lme']<=row['sz30']:
        value='S'
    elif row['lme']<=row['sz70']:
        value='M'
    elif row['lme']>row['sz70']:
        value='L'
    else:
        value=''
    return value

def perf_bucket(row):
    if 0<=row['lperf']<=row['perf30']:
        value = 'L'
    elif row['lperf']<=row['perf70']:
        value='M'
    elif row['lperf']>row['perf70']:
        value='H'
    else:
        value=''
    return value

In [12]:
data1['szport']=np.where((data1['lperf']>0)&(data1['lme']>0)&(data1['count']>=1), data1.apply(sz_bucket, axis=1), '')
data1['perfport']=np.where((data1['lperf']>0)&(data1['lme']>0)&(data1['count']>=1), data1.apply(perf_bucket, axis=1), '')

In [13]:
# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan

In [14]:
# value-weighted return
vwret=data1.groupby(['jdate','szport','perfport']).apply(wavg, 'retadj','wt').to_frame().reset_index().rename(columns={0: 'vwret'})
vwret['spport']=vwret['szport']+vwret['perfport']

# firm count
vwret_n=data1.groupby(['jdate','szport','perfport'])['retadj'].count().reset_index().rename(columns={'retadj':'n_firms'})
vwret_n['spport']=vwret_n['szport']+vwret_n['perfport']

# transpose
ff_factors=vwret.pivot(index='jdate', columns='spport', values='vwret').reset_index()
ff_nfirms=vwret_n.pivot(index='jdate', columns='spport', values='n_firms').reset_index()

In [15]:
ff_factors.head(20)

spport,jdate,LH,LL,LM,MH,ML,MM,SH,SL,SM
0,1978-01-31,-0.048296,-0.052969,-0.062431,-0.038536,-0.036053,-0.043275,-0.020230,-0.015551,-0.022158
1,1978-02-28,-0.024221,-0.011740,-0.013550,0.002567,-0.003043,-0.000840,0.027677,0.011917,0.012042
2,1978-03-31,0.015836,0.038717,0.026506,0.085692,0.043436,0.069494,0.078661,0.060633,0.074872
3,1978-04-30,0.109471,0.063915,0.092760,0.089387,0.041694,0.078268,0.090499,0.067469,0.078590
4,1978-05-31,0.040954,-0.000693,0.012122,0.055203,0.035113,0.044097,0.084904,0.048693,0.066225
5,1978-06-30,-0.015349,-0.025433,-0.018430,-0.003906,-0.002185,-0.015053,-0.010576,0.008950,-0.005952
6,1978-07-31,0.075600,0.045824,0.064391,0.069207,0.045870,0.063914,0.059439,0.044679,0.041640
7,1978-08-31,0.044396,0.026162,0.028220,0.068758,0.043053,0.077466,0.102194,0.055533,0.104545
8,1978-09-30,-0.023752,0.013174,-0.002201,-0.025409,0.000641,-0.012970,-0.012901,0.003202,0.010941
9,1978-10-31,-0.114100,-0.086539,-0.078396,-0.187589,-0.125606,-0.152317,-0.236629,-0.204976,-0.209577


In [16]:
# create SMB and HML factors
ff_factors['WH']=(ff_factors['LH']+ff_factors['MH']+ff_factors['SH'])/3
ff_factors['WLo']=(ff_factors['LL']+ff_factors['ML']+ff_factors['SL'])/3
ff_factors['WHML'] = ff_factors['WH']-ff_factors['WLo']

ff_factors['WLa']=(ff_factors['LH']+ff_factors['LL']+ff_factors['LM'])/3
ff_factors['WS']=(ff_factors['SL']+ff_factors['SH']+ff_factors['SL'])/3
ff_factors['WSML'] = ff_factors['WS']-ff_factors['WLa']
ff_factors=ff_factors.rename(columns={'jdate':'date'})

# n firm count
ff_nfirms['H']=ff_nfirms['SH']+ff_nfirms['MH']+ff_nfirms['LH']
ff_nfirms['Lo']=ff_nfirms['SL']+ff_nfirms['ML']+ff_nfirms['LL']
ff_nfirms['HML']=ff_nfirms['H']+ff_nfirms['Lo']

ff_nfirms['La']=ff_nfirms['LH']+ff_nfirms['LL']+ff_nfirms['LM']
ff_nfirms['S']=ff_nfirms['SL']+ff_nfirms['SM']+ff_nfirms['SH']
ff_nfirms['SML']=ff_nfirms['La']+ff_nfirms['S']
ff_nfirms['TOTAL']=ff_nfirms['SML']
ff_nfirms=ff_nfirms.rename(columns={'jdate':'date'})

In [44]:
ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'date'}, inplace=True)
ff3_monthly.rename(columns={"Mkt-RF": 'Mkt_RF'}, inplace=True)
ff3_factors = pd.merge(ff3_monthly, ff_factors, how='inner', on=['date'])
ff3_factors.head()

ff_large = ff3_factors[['date', 'Mkt_RF', 'SMB', 'HML', 'RF', 'WSML']]
ff_large = ff_large.copy()
ff_large.head()

,date,Mkt_RF,SMB,HML,RF,WSML
0,1978-01-31,-0.0601,0.0224,0.0336,0.0049,0.037455
1,1978-02-28,-0.0138,0.0359,0.0083,0.0046,0.033674
2,1978-03-31,0.0285,0.0348,0.0116,0.0053,0.039623
3,1978-04-30,0.0788,0.0044,-0.0357,0.0054,-0.013570
4,1978-05-31,0.0176,0.0456,-0.0053,0.0051,0.043302


In [45]:
ff_large['WSML-RF'] = ff_large['WSML']-ff_large['RF']
model = smf.formula.ols(formula = "WSML-RF ~ Mkt_RF + SMB + HML", data = ff_large).fit()
print(model.params)

Intercept    0.000765
Mkt_RF      -0.001078
SMB          0.937079
HML          0.118919
dtype: float64
